In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 
import pickle

import pandas as pd

In [ ]:


data = pd.read_pickle('pickles/data_with_punctuation.pkl')
data

In [ ]:
data.index

In [ ]:
data.transcript.loc['AnaA']

In [ ]:
def pickleData(info, nameWrite, tipo):
    pickle.dump(info, open(nameWrite+'.pkl', "wb"))

In [ ]:
"""def createCorpus(data,nameWrite,tipo):
    corpus = data.split(".")
    while("" in corpus) : 
        corpus.remove("")
    pickleData(corpus,'generation/'+nameWrite+'_reviews_sentences', tipo)"""


In [ ]:
"""createCorpus(data.transcript.loc['JeriR'], "JeriR", "wb")"""

In [ ]:
"""for actor in data.index:
    createCorpus(data.transcript.loc[actor], actor, "wb")"""

In [ ]:
"""corpus =  pd.read_pickle('generation/JeriR_reviews_sentences.pkl')
corpus"""

In [ ]:
"""for actor in data.index:
    createCorpus(data.transcript.loc[actor], "all", "ab")"""

In [ ]:
corpus =  pd.read_pickle('generation/all_reviews_sentences.pkl')

In [ ]:
len(corpus)

In [ ]:
corpus

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1


print(total_words)


In [ ]:
print(tokenizer.word_index)

In [ ]:
"""from nltk.corpus import stopwords
stops = set(stopwords.words("english"))      """            

"""def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)"""

#imdb['stem_meaningful'] = imdb.apply(remove_stops, axis=1)
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    print(token_list)

In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len = 100
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# creamos en conjunto de datos de entrenamiento y sus salidas esperadas
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
xs

In [ ]:
ys[10]

In [ ]:
print(tokenizer.word_index['open'])
print(tokenizer.word_index['movie'])
print(tokenizer.word_index['love'])
print(tokenizer.word_index['of'])
print(tokenizer.word_index['in'])
print(tokenizer.word_index['i'])
print(tokenizer.word_index['feel'])

In [ ]:
#print(xs[2])

In [ ]:
#print(ys[2])

In [ ]:
print("total_words: ",total_words, "| max_sequence_len: ", max_sequence_len)


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=15, verbose=1, batch_size=10)

print(model.summary())



In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
seed_text = "I really hate"
next_words = 27
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)